<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/files_2020/cce-color.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Pipeline de Datos de Contratación Pública
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>

## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Junio 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Pipeline de Datos de Contratación Pública**|
|**Descripción y alcance**|Notebook para la limpieza y validación de los datos de la base unificada.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos -GAEC</td></tr><tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Ricardo Suarez</td></tr> <tr><td>*Cargo:*</td> <td>Subdirector Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

# Introducción

En este notebook, nos centraremos en la limpieza y preprocesamiento de los datos obtenidos de la base de contratos del Sistema Electrónico para la Contratación Pública (SECOP) I y II de Colombia. Los conjuntos de datos con los que estamos trabajando contienen una variedad de información sobre contratos públicos, desde sus detalles operativos hasta sus atributos financieros.

El objetivo principal de este ejercicio es preparar estos conjuntos de datos para análisis posteriores. Esta preparación incluye una serie de pasos que asegurarán que los datos estén en el formato correcto y sean consistentes, comprensibles y confiables.

Cargamos la base previamente descargada y con las variables unificadas entre los conjuntos de datos de cada fuente para asegurar que la misma información se represente de manera coherente en los mismos. Esto puede incluir el mapeo de categorías equivalentes entre los conjuntos de datos y la armonización entre variables y sus nombres.

A continuación, estandarizaremos los datos para garantizar que todas las variables se presenten en un formato uniforme. Este proceso puede involucrar la normalización de texto, la conversión de datos categóricos en formatos numéricos y la manipulación de fechas y tiempos para que se presenten de manera consistente.

Una parte crucial de este trabajo implica limpiar los datos de elementos no deseados. Esto implica quitar espacios extra, eliminar caracteres especiales y, en general, asegurarnos de que los datos sean lo más limpios y precisos posible.

Además, verificaremos los tipos de variables en nuestros conjuntos de datos para asegurarnos de que sean apropiados para el tipo de datos que contienen. Esto podría implicar la conversión de datos numéricos almacenados como texto en números reales, o viceversa.

Comprobaremos si hay registros duplicados en nuestros conjuntos de datos. Si existen, los almacenaremos para realizar la validación y evitar cualquier sesgo o inexactitud en los análisis posteriores.

Finalmente, validaremos la cantidad de valores nulos y los valores negativos en la variable de cantidad de contrato. En el caso de los valores nulos, decidiremos si tienen coherencia, dependiendo del contexto. Los valores negativos, si no tienen sentido en el contexto de nuestro análisis, serán almacenados y validados desde la descarga de los datos.

Al final de este proceso, tendremos un conjunto de datos limpio y de alta calidad que estará listo para ser utilizado en futuros análisis y modelado de datos.

## Cargue del archivo parquet

### Cargue de librerías

En esta sección se cargan las librerías necesarias para el desarrollo del script y se establecen los parámetros de trabajo.

In [1]:
### Paquetes usados para la exploración de datos

import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import unidecode
import libreria.data_management as dm
from dataprep.clean import clean_duplication
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Cargamos la ruta de los archivos
SECOP_bases = ['SECOP_2020', 'SECOP_2021']
df_SECOP_bases = {}

for base in SECOP_bases:
    df_SECOP_bases[base] = pd.read_csv(f"../../../muestras de datos/procesados/{base}.csv", sep=';')
    df_SECOP_bases[base] = df_SECOP_bases[base].drop('index', axis=1)

Validación tipos de variables

In [5]:
for df_datos in df_SECOP_bases:
    df_SECOP_bases[df_datos]['Fuente'] = df_SECOP_bases[df_datos]['Fuente'].astype(str)
    df_SECOP_bases[df_datos]['ID_Contrato'] = df_SECOP_bases[df_datos]['ID_Contrato'].astype(str)
    df_SECOP_bases[df_datos]['ID_Proceso'] = df_SECOP_bases[df_datos]['ID_Proceso'].astype(str)
    df_SECOP_bases[df_datos]['ID_Entidad'] = df_SECOP_bases[df_datos]['ID_Entidad'].astype(str)
    df_SECOP_bases[df_datos]['Nombre_Entidad'] = df_SECOP_bases[df_datos]['Nombre_Entidad'].astype(str)
    df_SECOP_bases[df_datos]['NIT_Entidad'] = df_SECOP_bases[df_datos]['NIT_Entidad'].astype(str)
    df_SECOP_bases[df_datos]['Orden_Entidad'] = df_SECOP_bases[df_datos]['Orden_Entidad'].astype(str)
    df_SECOP_bases[df_datos]['Modalidad'] = df_SECOP_bases[df_datos]['Modalidad'].astype(str)
    df_SECOP_bases[df_datos]['Estado'] = df_SECOP_bases[df_datos]['Estado'].astype(str)
    df_SECOP_bases[df_datos]['Descripcion_proceso'] = df_SECOP_bases[df_datos]['Descripcion_proceso'].astype(str)
    df_SECOP_bases[df_datos]['Objeto_Contrato'] = df_SECOP_bases[df_datos]['Objeto_Contrato'].astype(str)
    df_SECOP_bases[df_datos]['Tipo_de_contrato'] = df_SECOP_bases[df_datos]['Tipo_de_contrato'].astype(str)
    df_SECOP_bases[df_datos]['UNSPSC'] = df_SECOP_bases[df_datos]['UNSPSC'].astype(str)
    df_SECOP_bases[df_datos]['Nombre_Proveedor'] = df_SECOP_bases[df_datos]['Nombre_Proveedor'].astype(str)
    df_SECOP_bases[df_datos]['Documento_Proveedor'] = df_SECOP_bases[df_datos]['Documento_Proveedor'].astype(str)
    df_SECOP_bases[df_datos]['Tipo_proveedor'] = df_SECOP_bases[df_datos]['Tipo_proveedor'].astype(str)
    df_SECOP_bases[df_datos]['Departamento_Entidad'] = df_SECOP_bases[df_datos]['Departamento_Entidad'].astype(str)
    df_SECOP_bases[df_datos]['Municipio_Entidad'] = df_SECOP_bases[df_datos]['Municipio_Entidad'].astype(str)
    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = df_SECOP_bases[df_datos]['Departamento_Proveedor'].astype(str)
    df_SECOP_bases[df_datos]['Municipio_Proveedor'] = df_SECOP_bases[df_datos]['Municipio_Proveedor'].astype(str)
    df_SECOP_bases[df_datos]['Link'] = df_SECOP_bases[df_datos]['Link'].astype(str)
    # df_datos['Modalidad Corta'] = df_datos['Modalidad Corta'].astype(str)
    
    df_SECOP_bases[df_datos]['Fecha_firma'] = pd.to_datetime(df_SECOP_bases[df_datos]['Fecha_firma'])
    df_SECOP_bases[df_datos]['Fecha_inicio_contrato'] = pd.to_datetime(df_SECOP_bases[df_datos]['Fecha_inicio_contrato'])
    
    df_SECOP_bases[df_datos]['Valor_contrato'] = df_SECOP_bases[df_datos]['Valor_contrato'].astype('float64')

df_no_convertidos = pd.DataFrame()
for valor in df_SECOP_bases[df_datos]['Fecha_fin_contrato']:
    try:
        pd.to_datetime(valor)
    except:
        df_no_convertidos = df_no_convertidos.append({'Fecha_fin_contrato': valor}, ignore_index=True)

In [6]:
df_SECOP_bases['SECOP_2020'].dtypes

ID_Contrato                       object
ID_Proceso                        object
ID_Entidad                        object
Nombre_Entidad                    object
NIT_Entidad                       object
Orden_Entidad                     object
Modalidad                         object
Estado                            object
Descripcion_proceso               object
Objeto_Contrato                   object
Tipo_de_contrato                  object
Fecha_firma               datetime64[ns]
UNSPSC                            object
Nombre_Proveedor                  object
Documento_Proveedor               object
Tipo_proveedor                    object
Valor_contrato                   float64
Departamento_Entidad              object
Municipio_Entidad                 object
Departamento_Proveedor            object
Municipio_Proveedor               object
Fecha_inicio_contrato     datetime64[ns]
Fecha_fin_contrato                object
Link                              object
Fuente          

Tratamiento de espacios y caracteres especiales

In [10]:
for df_datos in df_SECOP_bases:
    df_SECOP_bases[df_datos]['Fuente'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Fuente'])
    df_SECOP_bases[df_datos]['Nombre_Entidad'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Nombre_Entidad'])
    df_SECOP_bases[df_datos]['Orden_Entidad'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Orden_Entidad'])
    df_SECOP_bases[df_datos]['Modalidad'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Modalidad'])
    df_SECOP_bases[df_datos]['Estado'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Estado'])
    df_SECOP_bases[df_datos]['Descripcion_proceso'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Descripcion_proceso'])
    df_SECOP_bases[df_datos]['Objeto_Contrato'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Objeto_Contrato'])
    df_SECOP_bases[df_datos]['Tipo_de_contrato'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Tipo_de_contrato'])
    df_SECOP_bases[df_datos]['Departamento_Entidad'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Departamento_Entidad'])
    df_SECOP_bases[df_datos]['Nombre_Proveedor'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Nombre_Proveedor'])
    df_SECOP_bases[df_datos]['Tipo_proveedor'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Tipo_proveedor'])
    df_SECOP_bases[df_datos]['Municipio_Entidad'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Municipio_Entidad'])
    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Departamento_Proveedor'])
    df_SECOP_bases[df_datos]['Municipio_Proveedor'] = dm.unificar_series(serie=df_SECOP_bases[df_datos]['Municipio_Proveedor'])

Tratamiento de espacios con strip

In [11]:
for df_datos in df_SECOP_bases:
    df_SECOP_bases[df_datos]['ID_Contrato'] = df_SECOP_bases[df_datos]['ID_Contrato'].str.strip()
    df_SECOP_bases[df_datos]['ID_Proceso'] = df_SECOP_bases[df_datos]['ID_Proceso'].str.strip()
    df_SECOP_bases[df_datos]['ID_Entidad'] = df_SECOP_bases[df_datos]['ID_Entidad'].str.strip()
    df_SECOP_bases[df_datos]['NIT_Entidad'] = df_SECOP_bases[df_datos]['NIT_Entidad'].str.strip()
    df_SECOP_bases[df_datos]['UNSPSC'] = df_SECOP_bases[df_datos]['UNSPSC'].str.strip()
    df_SECOP_bases[df_datos]['Documento_Proveedor'] = df_SECOP_bases[df_datos]['Documento_Proveedor'].str.strip()

Verificar duplicados en contratos

In [12]:
df_SECOP_bases[df_datos]['ID_Contrato'].value_counts().reset_index().head(30)

,index,ID_Contrato
0,11495788,100
1,10996036,100
2,11138697,90
3,11227834,90
4,11061963,90
5,11212879,80
6,11216296,80
7,11138508,80
8,11085056,80
9,10876512,80


In [13]:
duplicados = df_SECOP_bases[df_datos]['ID_Contrato'].value_counts().reset_index()

In [14]:
print('Cantidad de contratos duplicados:')
print(len(duplicados[(duplicados['ID_Contrato']>1)]))
duplicados[(duplicados['ID_Contrato']>1)]

Cantidad de contratos duplicados:
195259


,index,ID_Contrato
0,11495788,100
1,10996036,100
2,11138697,90
3,11227834,90
4,11061963,90
...,...,...
195254,11277794,2
195255,11288226,2
195256,11159314,2
195257,11373441,2


Detectar negativos en valor del contrato

In [15]:
print('Cantidad de contratos con valor negativo:')
print(len(df_SECOP_bases[df_datos][(df_SECOP_bases[df_datos]['Valor_contrato']<0)]))
print('ID de los contratos:')
print(df_SECOP_bases[df_datos][(df_SECOP_bases[df_datos]['Valor_contrato']<0)]['ID_Contrato'].unique())
df_SECOP_bases[df_datos][(df_SECOP_bases[df_datos]['Valor_contrato']<0)]

Cantidad de contratos con valor negativo:
0
ID de los contratos:
[]


,ID_Contrato,ID_Proceso,ID_Entidad,Nombre_Entidad,NIT_Entidad,Orden_Entidad,Modalidad,Estado,Descripcion_proceso,Objeto_Contrato,...,Tipo_proveedor,Valor_contrato,Departamento_Entidad,Municipio_Entidad,Departamento_Proveedor,Municipio_Proveedor,Fecha_inicio_contrato,Fecha_fin_contrato,Link,Fuente


Aplicación de la función remove_extra_punct() del script de data_management que unifica variables y remueve caracteres especiales.

In [17]:
for df_datos in df_SECOP_bases:
    df_SECOP_bases[df_datos]['Fuente'] = df_SECOP_bases[df_datos]['Fuente'].apply(dm.remove_extra_punct)
    df_SECOP_bases[df_datos]['Nombre_Entidad'] = df_SECOP_bases[df_datos]['Nombre_Entidad'].apply(dm.remove_extra_punct) # TODO
    df_SECOP_bases[df_datos]['Orden_Entidad'] = df_SECOP_bases[df_datos]['Orden_Entidad'].apply(dm.remove_extra_punct) # TODO
    df_SECOP_bases[df_datos]['Modalidad'] = df_SECOP_bases[df_datos]['Modalidad'].apply(dm.remove_extra_punct)
    df_SECOP_bases[df_datos]['Estado'] = df_SECOP_bases[df_datos]['Estado'].apply(dm.remove_extra_punct)
    df_SECOP_bases[df_datos]['Descripcion_proceso'] = df_SECOP_bases[df_datos]['Descripcion_proceso'].apply(dm.remove_extra_punct)
    df_SECOP_bases[df_datos]['Objeto_Contrato'] = df_SECOP_bases[df_datos]['Objeto_Contrato'].apply(dm.remove_extra_punct)
    df_SECOP_bases[df_datos]['Tipo_de_contrato'] = df_SECOP_bases[df_datos]['Tipo_de_contrato'].apply(dm.remove_extra_punct) # TODO
    df_SECOP_bases[df_datos]['Departamento_Entidad'] = df_SECOP_bases[df_datos]['Departamento_Entidad'].apply(dm.remove_extra_punct) # TODO
    df_SECOP_bases[df_datos]['Nombre_Proveedor'] = df_SECOP_bases[df_datos]['Nombre_Proveedor'].apply(dm.remove_extra_punct) # TODO
    df_SECOP_bases[df_datos]['Tipo_proveedor'] = df_SECOP_bases[df_datos]['Tipo_proveedor'].apply(dm.remove_extra_punct) # TODO
    df_SECOP_bases[df_datos]['Municipio_Entidad'] = df_SECOP_bases[df_datos]['Municipio_Entidad'].apply(dm.remove_extra_punct) # TODO
    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = df_SECOP_bases[df_datos]['Departamento_Proveedor'].apply(dm.remove_extra_punct) # TODO
    df_SECOP_bases[df_datos]['Municipio_Proveedor'] = df_SECOP_bases[df_datos]['Municipio_Proveedor'].apply(dm.remove_extra_punct) # TODO

In [32]:
# Arreglos por modalidad
for df_datos in df_SECOP_bases:
    df_SECOP_bases[df_datos]["Modalidad Estandar"] = df_SECOP_bases[df_datos]['Modalidad'].apply(dm.Tipo_Proceso)
    df_SECOP_bases[df_datos]["Grupo Modalidad"] = df_SECOP_bases[df_datos]["Modalidad Estandar"].apply(dm.Grupo_Modalidad)

Se unifican las varibles departamento_entidad, municipio_entidad, departamento_proveedor y municipio_proveedor

In [36]:
columns = ['Departamento_Entidad', 'Municipio_Entidad', 'Departamento_Proveedor', 'Municipio_Proveedor']
# Se arreglan los Nombres Geograficos
# Se usa BOGOTA D.C.
# Se arregla NARIÑO
# ==============================================================================
replacement_mapping_dict = {"DISTRITO CAPITAL DE BOGOTA": "BOGOTA D.C.", 
                            "BOGOTA D.C.": "BOGOTA D.C.",
                            "NARINO" : "NARIÑO"}

for column in columns:
    for df_datos in df_SECOP_bases:
        df_SECOP_bases[df_datos][column] = df_SECOP_bases[df_datos][column].str.upper()

        # Se llenan los campos vacíos con "No Definido"
        df_SECOP_bases[df_datos][column] = df_SECOP_bases[df_datos][column].fillna("No Definido")

        # Se limpian los nombres de tildes y caracteres especiales
        df_SECOP_bases[df_datos][column] = df_SECOP_bases[df_datos][column].apply(unidecode.unidecode)

        df_SECOP_bases[df_datos][column] = df_SECOP_bases[df_datos][column].replace(replacement_mapping_dict)

Se aplica la función clean_duplication de dataprep:

In [ ]:
for df_datos in df_SECOP_bases:
    clean_duplication(df_SECOP_bases[df_datos], "Modalidad")

Se ven los primeros registros del dataframe

In [37]:
df_SECOP_bases['SECOP_2020'].head()

,ID_Contrato,ID_Proceso,ID_Entidad,Nombre_Entidad,NIT_Entidad,Orden_Entidad,Modalidad,Estado,Descripcion_proceso,Objeto_Contrato,Tipo_de_contrato,Fecha_firma,UNSPSC,Nombre_Proveedor,Documento_Proveedor,Tipo_proveedor,Valor_contrato,Departamento_Entidad,Municipio_Entidad,Departamento_Proveedor,Municipio_Proveedor,Fecha_inicio_contrato,Fecha_fin_contrato,Link,Fuente,Modalidad_Estandar,Grupo_Modalidad
0,9575722,19-1-206746,215001021.0,boyaca empresa departamental servicios publico...,900297725,territorial,licitacion publica,liquidado,construccin plan maestro alcantarillado munici...,construccin plan maestro alcantarillado munici...,obra,2020-01-24,72141100.0,consorcio sas,900297725,,1.462354e+09,BOYACA,TUNJA,BOYACA,BOYACA,2020-01-24,2020-10-24,https://www.contratos.gov.co/consultas/detalle...,secop i,Licitación Pública,Modalidad Competitiva
1,9575722,19-1-206746,215001021.0,boyaca empresa departamental servicios publico...,900297725,territorial,licitacion publica,liquidado,construccin plan maestro alcantarillado munici...,construccin plan maestro alcantarillado munici...,obra,2020-01-24,72141100.0,consorcio sas,900297725,,1.462354e+09,BOYACA,TUNJA,BOYACA,BOYACA,2020-01-24,2020-10-24,https://www.contratos.gov.co/consultas/detalle...,secop i,Licitación Pública,Modalidad Competitiva
2,9575722,19-1-206746,215001021.0,boyaca empresa departamental servicios publico...,900297725,territorial,licitacion publica,liquidado,construccin plan maestro alcantarillado munici...,construccin plan maestro alcantarillado munici...,obra,2020-01-24,72141100.0,consorcio sas,900297725,,1.462354e+09,BOYACA,TUNJA,BOYACA,BOYACA,2020-01-24,2020-10-24,https://www.contratos.gov.co/consultas/detalle...,secop i,Licitación Pública,Modalidad Competitiva
3,9575722,19-1-206746,215001021.0,boyaca empresa departamental servicios publico...,900297725,territorial,licitacion publica,liquidado,construccin plan maestro alcantarillado munici...,construccin plan maestro alcantarillado munici...,obra,2020-01-24,72141100.0,consorcio sas,900297725,,1.462354e+09,BOYACA,TUNJA,BOYACA,BOYACA,2020-01-24,2020-10-24,https://www.contratos.gov.co/consultas/detalle...,secop i,Licitación Pública,Modalidad Competitiva
4,9575722,19-1-206746,215001021.0,boyaca empresa departamental servicios publico...,900297725,territorial,licitacion publica,liquidado,construccin plan maestro alcantarillado munici...,construccin plan maestro alcantarillado munici...,obra,2020-01-24,72141100.0,consorcio sas,900297725,,1.462354e+09,BOYACA,TUNJA,BOYACA,BOYACA,2020-01-24,2020-10-24,https://www.contratos.gov.co/consultas/detalle...,secop i,Licitación Pública,Modalidad Competitiva


In [38]:
df_SECOP_bases['SECOP_2021'].head()

,ID_Contrato,ID_Proceso,ID_Entidad,Nombre_Entidad,NIT_Entidad,Orden_Entidad,Modalidad,Estado,Descripcion_proceso,Objeto_Contrato,Tipo_de_contrato,Fecha_firma,UNSPSC,Nombre_Proveedor,Documento_Proveedor,Tipo_proveedor,Valor_contrato,Departamento_Entidad,Municipio_Entidad,Departamento_Proveedor,Municipio_Proveedor,Fecha_inicio_contrato,Fecha_fin_contrato,Link,Fuente,Modalidad_Estandar,Grupo_Modalidad
0,5265886,16-12-5208719,215759016.0,boyaca instituto fomento recreacion deporte so...,826000058,territorial,contratacion directa ( ley ),celebrado,realizar organizacin desarrollo fase municipal...,realizar organizacin desarrollo fase municipal...,prestacion servicios,2021-05-27,94121500.0,fundacin cultural suamox,900426529,,4.449400e+07,BOYACA,SOGAMOSO,BOYACA,BOYACA,2016-05-27,2016-07-27,https://www.contratos.gov.co/consultas/detalle...,secop i,Contratación Directa,Modalidad Directa
1,8514015,19-4-9355355,25235636.0,nariño unimos empresa municipal telecomunicaci...,900292948,territorial,regimen especial,celebrado,prestacin servicios mantenimientos actualizaci...,prestacin servicios mantenimientos actualizaci...,prestacion servicios,2021-04-23,111500.0,siti soluciones sas,9005626021,,1.200000e+07,NARIÑO,IPIALES,NARIÑO,NARIÑO,2019-05-03,2020-01-03,https://www.contratos.gov.co/consultas/detalle...,secop i,Régimen Especial,Modalidad Especial
2,10616218,19-4-9598965,205001151.0,antioquia agencia educacion superior medellin ...,900602106,territorial,regimen especial,celebrado,convenio interadministrativo coordinar gestion...,ejecutar convenio marco interadministrativono ...,prestacion servicios,2021-01-28,86121700.0,instituto tecnolgico metropolitano itm,800214750,,6.622851e+09,ANTIOQUIA,MEDELLIN,ANTIOQUIA,ANTIOQUIA,2021-01-30,2024-08-30,https://www.contratos.gov.co/consultas/detalle...,secop i,Régimen Especial,Modalidad Especial
3,10616218,19-4-9598965,205001151.0,antioquia agencia educacion superior medellin ...,900602106,territorial,regimen especial,celebrado,convenio interadministrativo coordinar gestion...,ejecutar convenio marco interadministrativono ...,prestacion servicios,2021-01-28,86121700.0,instituto tecnolgico metropolitano itm,800214750,,6.622851e+09,ANTIOQUIA,MEDELLIN,ANTIOQUIA,ANTIOQUIA,2021-01-30,2024-08-30,https://www.contratos.gov.co/consultas/detalle...,secop i,Régimen Especial,Modalidad Especial
4,10616218,19-4-9598965,205001151.0,antioquia agencia educacion superior medellin ...,900602106,territorial,regimen especial,celebrado,convenio interadministrativo coordinar gestion...,ejecutar convenio marco interadministrativono ...,prestacion servicios,2021-01-28,86121700.0,instituto tecnolgico metropolitano itm,800214750,,6.622851e+09,ANTIOQUIA,MEDELLIN,ANTIOQUIA,ANTIOQUIA,2021-01-30,2024-08-30,https://www.contratos.gov.co/consultas/detalle...,secop i,Régimen Especial,Modalidad Especial


In [39]:
# Se revisa cantidad de nulos de cada variable

df_SECOP_bases['SECOP_2020'].isnull().sum()

ID_Contrato                  0
ID_Proceso                   0
ID_Entidad                   0
Nombre_Entidad               0
NIT_Entidad                  0
Orden_Entidad                0
Modalidad                    0
Estado                       0
Descripcion_proceso          0
Objeto_Contrato              0
Tipo_de_contrato             0
Fecha_firma                  0
UNSPSC                       0
Nombre_Proveedor             0
Documento_Proveedor          0
Tipo_proveedor               0
Valor_contrato               0
Departamento_Entidad         0
Municipio_Entidad            0
Departamento_Proveedor       0
Municipio_Proveedor          0
Fecha_inicio_contrato     5847
Fecha_fin_contrato          12
Link                         0
Fuente                       0
Modalidad_Estandar           0
Grupo_Modalidad              0
dtype: int64

In [40]:
# Se revisa cantidad de nulos de cada variable

df_SECOP_bases['SECOP_2021'].isnull().sum()

ID_Contrato                  0
ID_Proceso                   0
ID_Entidad                   0
Nombre_Entidad               0
NIT_Entidad                  0
Orden_Entidad                0
Modalidad                    0
Estado                       0
Descripcion_proceso          0
Objeto_Contrato              0
Tipo_de_contrato             0
Fecha_firma                  0
UNSPSC                       0
Nombre_Proveedor             0
Documento_Proveedor          0
Tipo_proveedor               0
Valor_contrato               0
Departamento_Entidad         0
Municipio_Entidad            0
Departamento_Proveedor       0
Municipio_Proveedor          0
Fecha_inicio_contrato     9625
Fecha_fin_contrato           3
Link                         0
Fuente                       0
Modalidad_Estandar           0
Grupo_Modalidad              0
dtype: int64

In [41]:
# Arreglos extras de cadenas de texto o nombres geográficos
for df_datos in df_SECOP_bases:
    #df_SECOP_bases[df_datos]['ID Contrato'] = df_SECOP_bases[df_datos]['ID Contrato'].str.replace('.0','')
    df_SECOP_bases[df_datos]['ID_Entidad'] = df_SECOP_bases[df_datos]['ID_Entidad'].str.replace('.0','')
    df_SECOP_bases[df_datos]['Orden_Entidad'] = df_SECOP_bases[df_datos]['Orden_Entidad'].replace('nan',np.nan)
    df_SECOP_bases[df_datos]['Modalidad'] = df_SECOP_bases[df_datos]['Modalidad'].replace('nan',np.nan)
    df_SECOP_bases[df_datos]['Tipo_de_contrato'] = df_SECOP_bases[df_datos]['Tipo_de_contrato'].replace('nan',np.nan)
    df_SECOP_bases[df_datos]['Tipo_de_contrato'] = df_SECOP_bases[df_datos]['Tipo_de_contrato'].replace('',np.nan)
    df_SECOP_bases[df_datos]['Tipo_proveedor'] = df_SECOP_bases[df_datos]['Tipo_proveedor'].replace('nan',np.nan)
    df_SECOP_bases[df_datos]['Tipo_proveedor'] = df_SECOP_bases[df_datos]['Tipo_proveedor'].replace('',np.nan)

    df_SECOP_bases[df_datos]['Departamento_Entidad'] = df_SECOP_bases[df_datos]['Departamento_Entidad'].replace('BOGOTA D.C .','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Departamento_Entidad'] = df_SECOP_bases[df_datos]['Departamento_Entidad'].replace('DISTRITO CAPITAL BOGOTA','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Departamento_Entidad'] = df_SECOP_bases[df_datos]['Departamento_Entidad'].replace('NAN',np.nan)

    df_SECOP_bases[df_datos]['Municipio_Entidad'] = df_SECOP_bases[df_datos]['Municipio_Entidad'].replace('BOGOTA D.C .','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Municipio_Entidad'] = df_SECOP_bases[df_datos]['Municipio_Entidad'].replace('BOGOTA','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Municipio_Entidad'] = df_SECOP_bases[df_datos]['Municipio_Entidad'].replace('NAN',np.nan)

    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = df_SECOP_bases[df_datos]['Departamento_Proveedor'].replace('BOGOTA D.C .','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = df_SECOP_bases[df_datos]['Departamento_Proveedor'].replace('BOGOTA','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = df_SECOP_bases[df_datos]['Departamento_Proveedor'].replace('BOGOTA D.C','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = df_SECOP_bases[df_datos]['Departamento_Proveedor'].replace('BOGOTA DC','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = df_SECOP_bases[df_datos]['Departamento_Proveedor'].replace('BOGOTA DISTRITO CAPITAL','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = df_SECOP_bases[df_datos]['Departamento_Proveedor'].replace('BOGOTA! D.C .','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = df_SECOP_bases[df_datos]['Departamento_Proveedor'].replace('DISTRITO CAPITAL BOGOTA','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = df_SECOP_bases[df_datos]['Departamento_Proveedor'].replace('D.C','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Departamento_Proveedor'] = df_SECOP_bases[df_datos]['Departamento_Proveedor'].replace('NAN',np.nan)

    df_SECOP_bases[df_datos]['Municipio_Proveedor'] = df_SECOP_bases[df_datos]['Municipio_Proveedor'].replace('BOGOTA D.C .','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Municipio_Proveedor'] = df_SECOP_bases[df_datos]['Municipio_Proveedor'].replace('BOGOTA','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Municipio_Proveedor'] = df_SECOP_bases[df_datos]['Municipio_Proveedor'].replace('BOGOTA D.C','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Municipio_Proveedor'] = df_SECOP_bases[df_datos]['Municipio_Proveedor'].replace('BOGOTA DC','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Municipio_Proveedor'] = df_SECOP_bases[df_datos]['Municipio_Proveedor'].replace('BOGOTA DISTRITO CAPITAL','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Municipio_Proveedor'] = df_SECOP_bases[df_datos]['Municipio_Proveedor'].replace('BOGOTA! D.C .','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Municipio_Proveedor'] = df_SECOP_bases[df_datos]['Municipio_Proveedor'].replace('D.C','BOGOTA D.C.')
    df_SECOP_bases[df_datos]['Municipio_Proveedor'] = df_SECOP_bases[df_datos]['Municipio_Proveedor'].replace('NAN',np.nan)

Almacenamiento del archivo como CSV y como parquet

In [55]:
df_SECOP_bases['SECOP_2020'].to_csv('SECOP_2020_depurado_v3.csv', index=False, sep=';')
df_SECOP_bases['SECOP_2021'].to_csv('SECOP_2021_depurado_v3.csv', index=False, sep=';')

In [56]:
df_SECOP_bases['SECOP_2020'].to_parquet('../../../muestras de datos/procesados/SECOP_2020_depurado_v3.parquet', index=False, engine='fastparquet')
df_SECOP_bases['SECOP_2021'].to_parquet('../../../muestras de datos/procesados/SECOP_2021_depurado_v3.parquet', index=False, engine='fastparquet')

Se lee el archivo parquet almacenado

In [46]:
df_datos_2020 = pd.read_parquet('../../../muestras de datos/procesados/SECOP_2020_depurado_v3.parquet')
df_datos_2020.head()

,ID_Contrato,ID_Proceso,ID_Entidad,Nombre_Entidad,NIT_Entidad,Orden_Entidad,Modalidad,Estado,Descripcion_proceso,Objeto_Contrato,...,Departamento_Entidad,Municipio_Entidad,Departamento_Proveedor,Municipio_Proveedor,Fecha_inicio_contrato,Fecha_fin_contrato,Link,Fuente,Modalidad Estandar,Grupo Modalidad
0,9575722,19-1-206746,21021,boyaca empresa departamental servicios publico...,900297725,territorial,licitacion publica,liquidado,construccin plan maestro alcantarillado munici...,construccin plan maestro alcantarillado munici...,...,BOYACA,TUNJA,BOYACA,BOYACA,2020-01-24,2020-10-24,https://www.contratos.gov.co/consultas/detalle...,secop i,Licitación Pública,Modalidad Competitiva
1,9575722,19-1-206746,21021,boyaca empresa departamental servicios publico...,900297725,territorial,licitacion publica,liquidado,construccin plan maestro alcantarillado munici...,construccin plan maestro alcantarillado munici...,...,BOYACA,TUNJA,BOYACA,BOYACA,2020-01-24,2020-10-24,https://www.contratos.gov.co/consultas/detalle...,secop i,Licitación Pública,Modalidad Competitiva
2,9575722,19-1-206746,21021,boyaca empresa departamental servicios publico...,900297725,territorial,licitacion publica,liquidado,construccin plan maestro alcantarillado munici...,construccin plan maestro alcantarillado munici...,...,BOYACA,TUNJA,BOYACA,BOYACA,2020-01-24,2020-10-24,https://www.contratos.gov.co/consultas/detalle...,secop i,Licitación Pública,Modalidad Competitiva
3,9575722,19-1-206746,21021,boyaca empresa departamental servicios publico...,900297725,territorial,licitacion publica,liquidado,construccin plan maestro alcantarillado munici...,construccin plan maestro alcantarillado munici...,...,BOYACA,TUNJA,BOYACA,BOYACA,2020-01-24,2020-10-24,https://www.contratos.gov.co/consultas/detalle...,secop i,Licitación Pública,Modalidad Competitiva
4,9575722,19-1-206746,21021,boyaca empresa departamental servicios publico...,900297725,territorial,licitacion publica,liquidado,construccin plan maestro alcantarillado munici...,construccin plan maestro alcantarillado munici...,...,BOYACA,TUNJA,BOYACA,BOYACA,2020-01-24,2020-10-24,https://www.contratos.gov.co/consultas/detalle...,secop i,Licitación Pública,Modalidad Competitiva


# Conclusiones

Se realiza la limpieza de la base maestra, logrando el objetivo de tener un conjunto de datos limpio y de alta calidad que estará listo para ser utilizado en futuros análisis.

Se debe tener presente la alerta de valores duplicados en ID_Contrato, valores negativos en valor del contrato y valores nulos en variables donde se debe contar con la información (si los hay) para realizar las validaciones correspondientes.

El proceso se realiza con archivos descargados por año para agilizar la ejecución del notebook.

Finalmente, es importante recordar que la limpieza y el preprocesamiento de datos es un paso fundamental en cualquier proyecto de análisis de datos. La atención al detalle en esta etapa puede tener un impacto significativo en la calidad y la precisión de cualquier insight o modelo que se desarrolle en las etapas posteriores.